## Storing the Generated Image in Amazon S3 Bucket

In [3]:
import boto3
import json
import base64
import io
import uuid
from PIL import Image

# Set up the Amazon Bedrock client
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"
)

# Define the model ID
model_id = "amazon.nova-canvas-v1:0"

# Prepare the input prompt
prompt = "create me an image of indian flag which should be held by cricketer ms dhoni in indian cricket team jersey on the moon"

# Create the request payload
body = json.dumps({
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "height": 1024,
            "width": 1024,
            "cfgScale": 8.0,
            "seed": 0
        }
    })

accept = "application/json"
content_type = "application/json"

# Invoke the Amazon Bedrock model
response = bedrock_client.invoke_model(
    modelId=model_id,
    body=body,
    accept=accept, 
    contentType=content_type
)

# Process the response
result = json.loads(response["body"].read())

base64_image = result.get("images")[0]
base64_bytes = base64_image.encode('ascii')
image_bytes = base64.b64decode(base64_bytes)

image = Image.open(io.BytesIO(image_bytes))
image.show()

s3 = boto3.client("s3")
id = uuid.uuid4()

bucket_name = "blog-2-bucket-for-image-storage"
object_path = f"generated-images/{id}.png"

response = s3.put_object(
            Bucket=bucket_name,
            Key=object_path,
            Body=image_bytes,
            ContentType='image/jpeg'
        )

object_url = f"https://{bucket_name}.s3.us-east-1.amazonaws.com/{object_path}"
print(f"Generated Image is Stored Successfully at S3 location : {object_url}")

Generated Image is Stored Successfully at S3 location : https://blog-2-bucket-for-image-storage.s3.us-east-1.amazonaws.com/generated-images/aca33f19-c5e3-464a-8e95-08093960887b.png
